### Import libraries and Load Data

In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
path_df = "News_dataset.pickle"
with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [3]:
df.head()

File_Name                                            Content  Category  \
0   001.txt  Ad sales boost Time Warner profit\r\n\r\nQuart...  business   
1   002.txt  Dollar gains on Greenspan speech\r\n\r\nThe do...  business   
2   003.txt  Yukos unit buyer faces loan claim\r\n\r\nThe o...  business   
3   004.txt  High fuel prices hit BA's profits\r\n\r\nBriti...  business   
4   005.txt  Pernod takeover talk lifts Domecq\r\n\r\nShare...  business   

  Complete_Filename  id  News_length  
0  001.txt-business   1         2569  
1  002.txt-business   1         2257  
2  003.txt-business   1         1557  
3  004.txt-business   1         2421  
4  005.txt-business   1         1575

In [4]:
df.loc[3]['Content']

'High fuel prices hit BA\'s profits\r\n\r\nBritish Airways has blamed high fuel prices for a 40% drop in profits.\r\n\r\nReporting its results for the three months to 31 December 2004, the airline made a pre-tax profit of Â£75m ($141m) compared with Â£125m a year earlier. Rod Eddington, BA\'s chief executive, said the results were "respectable" in a third quarter when fuel costs rose by Â£106m or 47.3%. BA\'s profits were still better than market expectation of Â£59m, and it expects a rise in full-year revenues.\r\n\r\nTo help offset the increased price of aviation fuel, BA last year introduced a fuel surcharge for passengers.\r\n\r\nIn October, it increased this from Â£6 to Â£10 one-way for all long-haul flights, while the short-haul surcharge was raised from Â£2.50 to Â£4 a leg. Yet aviation analyst Mike Powell of Dresdner Kleinwort Wasserstein says BA\'s estimated annual surcharge revenues - Â£160m - will still be way short of its additional fuel costs - a predicted extra Â£250m. Tu

### 1. Text cleaning and preparation

In [5]:
#Text cleaning

df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [10]:
#Text Preparation
df['Content_Parsed_2']=df['Content_Parsed_1'].str.lower()
punctuations=list("?:!.,;")
for punct in punctuations:
    df['Content_Parsed_2']=df['Content_Parsed_2'].replace(punct,'')
df['Content_Parsed_3']=df['Content_Parsed_2'].replace("'s",'')

In [25]:
df.loc[3]['Content_Parsed_3']

"high fuel prices hit ba's profits british airways has blamed high fuel prices for a 40% drop in profits. reporting its results for the three months to 31 december 2004, the airline made a pre-tax profit of â£75m ($141m) compared with â£125m a year earlier. rod eddington, ba's chief executive, said the results were respectable in a third quarter when fuel costs rose by â£106m or 47.3%. ba's profits were still better than market expectation of â£59m, and it expects a rise in full-year revenues. to help offset the increased price of aviation fuel, ba last year introduced a fuel surcharge for passengers. in october, it increased this from â£6 to â£10 one-way for all long-haul flights, while the short-haul surcharge was raised from â£2.50 to â£4 a leg. yet aviation analyst mike powell of dresdner kleinwort wasserstein says ba's estimated annual surcharge revenues - â£160m - will still be way short of its additional fuel costs - a predicted extra â£250m. turnover for the quarter was up 4.3%

### Use any 1 method for Lemmatization

In [11]:

#Stemming and Lemmatization
nltk.download('punkt')
nltk.download('wordnet')

nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HIMANSHU\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HIMANSHU\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HIMANSHU\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [18]:
lemmatizer = WordNetLemmatizer()
nrows = len(df)
lemmatized_text_list = []
for row in range(nrows):
    lemmatized_text=[]
    text = df.loc[row]['Content_Parsed_3']
    text_words = text.split(" ")
    for word in text_words:
        lemmatized_text.append(lemmatizer.lemmatize(word,pos='v'))
 

    lemmatized_text = " ".join(lemmatized_text)
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

df['Content_Parsed_4'] = lemmatized_text_list   
        

In [19]:
df['Content_Parsed_4']

0       ad sales boost time warner profit quarterly pr...
1       dollar gain on greenspan speech the dollar hav...
2       yukos unit buyer face loan claim the owners of...
3       high fuel price hit ba's profit british airway...
4       pernod takeover talk lift domecq share in uk d...
                              ...                        
2220    bt program to beat dialler scam bt be introduc...
2221    spam e-mail tempt net shoppers computer users ...
2222    be careful how you code a new european directi...
2223    us cyber security chief resign the man make su...
2224    lose yourself in online game online role play ...
Name: Content_Parsed_4, Length: 2225, dtype: object

### Removing Stopwords (Any 1 Method)

In [20]:
#Downloading

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HIMANSHU\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
stop_list_final=[]
nrows = len(df)
stopwords_english = stopwords.words('english') 

for row in range(nrows):
    
    # Create an empty list containing no stop words
    stop_list = []
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to remove stopwords
    for word in text_words:
        if (word not in stopwords_english): 
          stop_list.append(word)
        
    # Join the list
    stop_text = " ".join(stop_list)
    # Append to the list containing the texts
    stop_list_final.append(stop_text)
df['Content_Parsed_5'] = stop_list_final  

In [27]:
df.loc[3]['Content_Parsed_5']

"high fuel price hit ba's profit british airways blame high fuel price 40% drop profits. report result three months 31 december 2004, airline make pre-tax profit â£75m ($141m) compare â£125m year earlier. rod eddington, ba's chief executive, say result respectable third quarter fuel cost rise â£106m 47.3%. ba's profit still better market expectation â£59m, expect rise full-year revenues. help offset increase price aviation fuel, ba last year introduce fuel surcharge passengers. october, increase â£6 â£10 one-way long-haul flights, short-haul surcharge raise â£2.50 â£4 leg. yet aviation analyst mike powell dresdner kleinwort wasserstein say ba's estimate annual surcharge revenues - â£160m - still way short additional fuel cost - predict extra â£250m. turnover quarter 4.3% â£1.97bn, benefit rise cargo revenue. look ahead full year result march 2005, ba warn yield - average revenues per passenger - expect decline continue lower price face competition low-cost carriers. however, say sales 

In [28]:
df.head(1)

File_Name                                            Content  Category  \
0   001.txt  Ad sales boost Time Warner profit\r\n\r\nQuart...  business   

  Complete_Filename  id  News_length  \
0  001.txt-business   1         2569   

                                    Content_Parsed_1  \
0  Ad sales boost Time Warner profit Quarterly pr...   

                                    Content_Parsed_2  \
0  ad sales boost time warner profit quarterly pr...   

                                    Content_Parsed_3  \
0  ad sales boost time warner profit quarterly pr...   

                                    Content_Parsed_4  \
0  ad sales boost time warner profit quarterly pr...   

                                    Content_Parsed_5  
0  ad sales boost time warner profit quarterly pr...

In [29]:
list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_5"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_5': 'Content_Parsed'})

In [30]:
df.head()

File_Name  Category Complete_Filename  \
0   001.txt  business  001.txt-business   
1   002.txt  business  002.txt-business   
2   003.txt  business  003.txt-business   
3   004.txt  business  004.txt-business   
4   005.txt  business  005.txt-business   

                                             Content  \
0  Ad sales boost Time Warner profit\r\n\r\nQuart...   
1  Dollar gains on Greenspan speech\r\n\r\nThe do...   
2  Yukos unit buyer faces loan claim\r\n\r\nThe o...   
3  High fuel prices hit BA's profits\r\n\r\nBriti...   
4  Pernod takeover talk lifts Domecq\r\n\r\nShare...   

                                      Content_Parsed  
0  ad sales boost time warner profit quarterly pr...  
1  dollar gain greenspan speech dollar hit highes...  
2  yukos unit buyer face loan claim owners embatt...  
3  high fuel price hit ba's profit british airway...  
4  pernod takeover talk lift domecq share uk drin...

### Label Encoding

In [31]:
#Generating new column for Category codes

category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}
# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})
     

In [32]:
df.head()

File_Name  Category Complete_Filename  \
0   001.txt  business  001.txt-business   
1   002.txt  business  002.txt-business   
2   003.txt  business  003.txt-business   
3   004.txt  business  004.txt-business   
4   005.txt  business  005.txt-business   

                                             Content  \
0  Ad sales boost Time Warner profit\r\n\r\nQuart...   
1  Dollar gains on Greenspan speech\r\n\r\nThe do...   
2  Yukos unit buyer faces loan claim\r\n\r\nThe o...   
3  High fuel prices hit BA's profits\r\n\r\nBriti...   
4  Pernod takeover talk lifts Domecq\r\n\r\nShare...   

                                      Content_Parsed  Category_Code  
0  ad sales boost time warner profit quarterly pr...              0  
1  dollar gain greenspan speech dollar hit highes...              0  
2  yukos unit buyer face loan claim owners embatt...              0  
3  high fuel price hit ba's profit british airway...              0  
4  pernod takeover talk lift domecq share uk drin...              0

In [33]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

### Text Representations

In [34]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [35]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1891, 300)
(334, 300)


In [36]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'business' category:
  . Most correlated unigrams:
. market
. economy
. oil
. growth
. bank
  . Most correlated bigrams:
. last year
. year old

# 'entertainment' category:
  . Most correlated unigrams:
. tv
. music
. award
. star
. film
  . Most correlated bigrams:
. mr blair
. prime minister

# 'politics' category:
  . Most correlated unigrams:
. minister
. blair
. election
. party
. labour
  . Most correlated bigrams:
. prime minister
. mr blair

# 'sport' category:
  . Most correlated unigrams:
. win
. side
. game
. team
. match
  . Most correlated bigrams:
. say mr
. year old

# 'tech' category:
  . Most correlated unigrams:
. digital
. technology
. computer
. software
. users
  . Most correlated bigrams:
. year old
. say mr



In [37]:
bigrams

['tell bbc', 'last year', 'prime minister', 'mr blair', 'year old', 'say mr']

In [39]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)

In [44]:
path_df1 = 'Pickles//y_test.pickle'
with open(path_df1, 'rb') as data:
    df1 = pickle.load(data)

In [45]:
df1

1691    3
1103    2
477     0
197     0
475     0
       ..
393     0
1746    3
444     0
2215    4
733     1
Name: Category_Code, Length: 334, dtype: int64